In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import eigsh
from scipy.sparse import identity as sid
%matplotlib inline

In [ ]:
def trap_method(a,b,yvals):
    Nm1 = yvals.size    
    return (b-a)/(Nm1) * np.sum(yvals)

**Problem 1**: (20pts) In quantum mechanics, it is really common to see boundary value problems of the form 

$$
-\epsilon\frac{d^{2}y}{dx^2} + \cos(\pi x)y = Ey, ~ y(-1) = y(1).
$$

where $0\leq \epsilon \ll 1$, i.e. we let $\epsilon$ be a small positive parameter.  $E\geq 0$ is the _energy_ of a particle trapped in an oscillating potential well $V(x) = \cos(\pi x)$, which is formed in crystal lattices of metals.  We likewise use _periodic-boundary conditions_ by setting 

$$
y(-1) = y(1).
$$

We desribe the probability of a particle being in the interval $[-1,a]$, $a<1$ via the formula

$$
P(-1\leq x \leq a) = \int_{-1}^{a} \tilde{y}(x) ~dx, ~ \tilde{y}(x) = \frac{y^{2}(x)}{\int_{-1}^{1}y^{2}(x)~dx}
$$

1a) (10pts) Using second-order centered-difference approximations and spdiags, write code which discretizes the operator $-\epsilon\frac{d^{2}y}{dx^2} + \cos(\pi x)y\approx A{\bf y}$, where ${\bf y}=\left(y_{1} ~y_{2}\cdots y_{N-1}\right)^{T}$, $y_{j}=y(x_{j})$.  Note, the periodic boundary conditions are implemented as 

$$
y_{0} = y_{N-1}, ~ y_{N} = y_{1}.
$$

This means that we have entries in the very top and the very bottom corners of our sparse matrix.  

1b) (5pts) You have now formed a discrete eigenvalue problem $A{\bf y} = E{\bf y}$.  Using the code below find the first 10 eigenvalues of the discretized equations.  For $\epsilon=1,.1,.01$ and $.001$, describe via a well designed plot how the first ten energy levels change as you decrease $\epsilon$.  

1c) (5pts) For $\epsilon=.01$, compare the associated probability distributions $\tilde{y}(x)$ for the first three energy levels.  How do the likelihoods of where a particle would be found change with changing energy?  Note, you'll need to use the Trapezoid Method (code included above) to compute the integrals used in defining $\tilde{y}(x)$.

In [ ]:
def eval_find(xvals,epvl):
    # xvals is your mesh.  epvl is your epsilon value.
    Np1 = xvals.size # this is N+1
    dx = # you add code here
    idx2 = # you add code here
    
    diag = # you add code here for the k=0 diagonal
    odiagp1 = # you add code here for the k=1 diagonal
    odiagn1 = # you add code here for the k=-1 diagonal
    odiagNp = # you add code here for the k=Np1-3 diagonal (the top corner)
    odiagNn = # you add code here for the k=-(Np1-3) diagonal (the bottom corner)
        
    # if you've everything correctly, this should just work from here on out
    data = np.array([odiagNn, odiagn1, diag, odiagp1, odiagNp])    
    dvals = np.array([-(Np1-3),-1,0,1,(Np1-3)])  
    Amat = spdiags(data, dvals, Np1-2, Np1-2)
    energy_values, ysols = eigsh(Amat,10,which='SM',mode='buckling') # return energies and corresponding y(x_j) solutions
   
    return energy_values, ysols

Nvls = 
xvals = np.linspace(-1.,1.,Nvls+1)

# This is just meant to get you started.  You'll need to think more about how to generate all the necessary plots.  
energy_values, ysols = eval_find(xvals,.1)
plt.scatter(np.arange(energy_values.size),energy_values[:])
plt.xlabel("$n$")
plt.ylabel("$E_{n}$") 

In [ ]:
# Plot ysols[:,0] which corresponds to energy_values[0]
plt.plot(xvals[1:-1],ysols[:,0]) 

**Problem 2**: 10 pts - For non-negative integer $n$, and $x\in[-1,1]$, define the function $T_{n}(x)=\cos(n\cos^{-1}(x))$.  
<ol>
        <li> (1pt) Show that $T_{0}(x)=1$
        <li> (1pt) Show that $T_{1}(x)=x$
        <li> (1pt) Show that $T_{2}(x)=2x^{2}-1$
        <li> (1pt) Show that $\sin(\cos^{-1}(x))=\sqrt{1-x^{2}}$            
        <li> (3pts) Using trignometric addition formulas, show that $T_{n+1}(x)=2xT_{n}(x) - T_{n-1}(x)$
        <li> (2pts) Using induction then, show that $T_{n}(x)$ is an $n^{th}$-degree polynomial.
</ol>

For this reason, we call $T_{n}(x)$ the _Chebyshev_ polynomials.  

**Problem 3**: 10 pts - For the Chebyshev points 

$$
x_{j} = \cos\left(\frac{2j+1}{2n+2}\pi\right), ~ j=0,\cdots,n
$$

<ol>
    <li> (3pts) Show that $T_{n+1}(x_{j})=0$.  
    <li> (3pt) Using the Fundamental Theorem of Algebra, show that 
        $$
        T_{n+1}(x) = c \prod_{j=0}^{n}(x-x_{j})
        $$
        where $c$ is some constant.  
    <li> (4pts) For the affiliated interpolating polynomial, say $p_{n}(x)$ where
        $$
        p_{n}(x) = \sum_{j=0}^{n}f_{j}L_{j}(x).
        $$
        Show using l'Hopital's rule and the fact that $L_{j}(x_{j})=1$ that  
        $$
        L_{j}(x) = \frac{T_{n+1}(x)}{(x-x_{j})T'_{n+1}(x_{j})}
        $$     
</ol>

**Problem 4**: 15 pts - Given $T_{n+1}(x)=\cos((n+1)\cos^{-1}(x))$:
<ol>
    <li> Show that (2pts)
        $$
        \frac{d}{dx}T_{n+1}(x) = (n+1)\frac{\sin((n+1)\cos^{-1}(x))}{\sqrt{1-x^{2}}}
        $$
    <li> Show then that (2pts)
        $$T'_{n+1}(x_{k}) = \frac{(n+1)(-1)^{k}}{\sqrt{1-x_{k}^{2}}}$$
    <li> Thus show that for $k\neq j$ that (2pts)
        $$L_{j}'(x_{k}) =   \frac{(-1)^{k+j}}{x_{k}-x_{j}}\sqrt{\frac{1-x_{j}^{2}}{1-x_{k}^{2}}}$$    
    <li> (Extra Credit +3, all or nothing) Using l'Hopital's rule, show (this can be quite tricky) that 
        $$
        \lim_{x\rightarrow x_{j}}L'_{j}(x) = \frac{T''(x_{j})}{2T'(x_{j})}
        $$
    <li> Thus show that (2pts)
        $$
        L'_{j}(x_{j}) = \frac{x_{j}}{2(1-x_{j}^{2})}
        $$
</ol>

Thus we now have the formula 
$$
L_{j}'(x_{k}) = \left\{
\begin{array}{lr}
\frac{(-1)^{k+j}}{x_{k}-x_{j}}\sqrt{\frac{1-x_{j}^{2}}{1-x_{k}^{2}}} & k\neq j\\
& \\
\frac{x_{j}}{2(1-x_{j}^{2})} & k=j
\end{array}
\right.
$$

so for given function $f(x)$, we can approximate $f'(x_{k})$ via the formula
$$
f'(x_{k})\approx p_{n}'(x_{k}) = \sum_{j=0}^{n}L'_{j}(x_{k})f_{j},
$$
which written in matrix/vector form is 
$$
\begin{pmatrix}f'(x_{0})\\f'(x_{1})\\\vdots\\f'(x_{n})\end{pmatrix} \approx {\bf D} \begin{pmatrix}f_{0}\\f_{1}\\\vdots\\f_{n}\end{pmatrix}, ~ {\bf D}_{kj} = L'_{j}(x_{k}).
$$
The code below builds ${\bf D}$ for you.  For the remainder of this problem, choose a test function $f(x)$, explain your choice, and then explore the error in using ${\bf D}$ to approximate the derivative of $f(x)$ for several choices of $N$.  Explain any trends or other features you observe in your experiments.  (7 pts)

In [ ]:
def build_dmat(cheb_pts):
    Np1 = cheb_pts.size
    Dmat = np.zeros((Np1,Np1))
    cvec = 1. - cheb_pts**2.
    Dmat += np.diag(cheb_pts/(2.*cvec))
    sclmat = np.sqrt(np.tile(1./cvec.reshape(-1,1),(1,Np1))*np.tile(cvec,(Np1,1)))
    for kk in range(Np1):
        for jj in range(kk+1, Np1):            
            dif = 1./(cheb_pts[kk]-cheb_pts[jj])
            Dmat[kk,jj] = (-1.)**(jj+kk) * dif * sclmat[kk,jj] 
            Dmat[jj,kk] = (-1.)**(jj+kk+1) * dif * sclmat[jj,kk]
    return Dmat

In [ ]:
ffun = lambda x: # pick a function to study and explain why you chose it 
dftrue = lambda x: # make sure you find the exact derivative of f 

In [ ]:
Npts = 
cheb_pts = np.cos( (2*np.arange(Npts+1)+1)/(2*Npts+2)*np.pi )
Dmat = build_dmat(cheb_pts)
# Test your derivative and explore the error for various values of Npts.  
# Make sure you refer to the Lecture_Nine notes for how to do matrix/vector products
# correctly.  